Carga el conjunto de datos MNIST y divídelo en un conjunto de entrenamiento, un conjunto de validación y un conjunto de prueba (por ejemplo, utiliza 50,000 instancias para el entrenamiento, 10,000 para la validación y 10,000 para las pruebas). 

In [27]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier, AdaBoostClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

In [4]:
from sklearn.datasets import fetch_openml

X_mnist, y_mnist = fetch_openml('mnist_784', return_X_y=True, as_frame=False, parser='auto')

Entrena clasificadores individuales, por ejemplo un clasificador RandomForest, un clasificador SVM y un MLP. 

In [5]:
# Los datos se normalizan dividiendo cada valor de píxel por 255.0,
# lo que escala los valores de píxeles al rango [0, 1]
X_mnist = X_mnist / 255.0

#El conjunto de datos MNIST se divide en tres conjuntos: entrenamiento (50,000 instancias),
# validación (10,000 instancias) y prueba (10,000 instancias).
X_train, y_train = X_mnist[:50000], y_mnist[:50000]
X_val, y_val = X_mnist[50000:60000], y_mnist[50000:60000]
X_test, y_test = X_mnist[60000:70000], y_mnist[60000:70000]

In [6]:
# Random Forest: 
# Un clasificador de bosques aleatorios con 500 estimadores
random_forest = RandomForestClassifier(n_estimators=500,
                                       bootstrap=True, 
                                       max_leaf_nodes=16,
                                       n_jobs=-1, 
                                       max_features='sqrt',
                                       oob_score=True,
                                       random_state=0)
random_forest.fit(X_train, y_train)

RandomForestClassifier(max_leaf_nodes=16, n_estimators=500, n_jobs=-1,
                       oob_score=True, random_state=0)

In [10]:
# Score del modelo de Random Forest
random_forest.oob_score_

0.81828

In [11]:
# Validamos el modelo
random_forest.score(X_val, y_val)

0.8395

In [12]:
# Testeamos el modelo
random_forest.score(X_test, y_test)

0.8315

SVM

In [28]:
# Un clasificador SVM
# con parámetros específicos de C y gamma.
svm_clf = SVC(C=0.2, gamma=0.6, random_state=0, tol=2)
svm_clf.fit(X_train, y_train)

SVC(C=0.2, gamma=0.6, random_state=0, tol=2)

In [14]:
# Validamos el modelo
svm_clf.score(X_val, y_val)

0.9505

In [15]:
# Testeamos el modelo
svm_clf.score(X_test, y_test)

0.9382

MLP

In [16]:
# Un clasificador de perceptrón multicapa
# con una capa oculta de 100 neuronas.
mlp = MLPClassifier(hidden_layer_sizes=(100, ), random_state=0)
mlp.fit(X_train, y_train)

MLPClassifier(random_state=0)

In [17]:

# Validación del modelo
mlp.score(X_val, y_val)

0.9777

In [18]:
# Testeamos el modelo
mlp.score(X_test, y_test)

0.9793

A continuación, intenta combinarlos en un ensamble cuyo desempeño supere a cada clasificador individual en el conjunto de validación, utilizando votación por mayoría hard y soft. Una vez que hayas encontrado uno, pruébalo en el conjunto de prueba.

Se crea un VotingClassifier con tres clasificadores (Random Forest, SVM y MLP) y se entrena primero con votación por mayoría (hard voting) y luego con votación suave (soft voting). Para la votación suave, se configura el clasificador SVM para que proporcione probabilidades de clase utilizando el atributo probability

In [19]:
# Crear un clasificador de votación por mayoría (hard voting)
voting_clf = VotingClassifier(
    estimators=[('rf', random_forest),
                ('svm', svm_clf), 
                ('mlp', mlp)],
    voting='hard',
    n_jobs=-1
)

# Entrenar el clasificador de votación
voting_clf.fit(X_train, y_train)




VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(max_leaf_nodes=16,
                                                     n_estimators=500,
                                                     n_jobs=-1, oob_score=True,
                                                     random_state=0)),
                             ('svm',
                              SVC(C=0.2, gamma=0.6, random_state=0, tol=2)),
                             ('mlp', MLPClassifier(random_state=0))],
                 n_jobs=-1)

In [20]:
# Validar el clasificador de votación en el conjunto de validación
voting_clf.score(X_val, y_val)

0.9657

In [22]:
# Cambiar a votación suave (soft voting) y configurar SVM para que tenga salida de probabilidad
voting_clf.voting = 'soft'
voting_clf.named_estimators['svm'].probability = True

n_cores = -1

voting_clf.n_jobs = n_cores

# Entrenar el clasificador de votación suave
voting_clf.fit(X_train, y_train)



VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(max_leaf_nodes=16,
                                                     n_estimators=500,
                                                     n_jobs=-1, oob_score=True,
                                                     random_state=0)),
                             ('svm',
                              SVC(C=0.2, gamma=0.6, probability=True,
                                  random_state=0, tol=2)),
                             ('mlp', MLPClassifier(random_state=0))],
                 n_jobs=-1, voting='soft')

In [23]:
# Probar el clasificador de votación suave en el conjunto de prueba
voting_clf.score(X_test, y_test)

0.979

Prueba otros ensambles usando Bagging, Boosting y Stacking (Stacking utiliza validación cruzada por lo que es mejor dividir los datos sólo en entrenamiento y prueba)

Se crea un clasificador Bagging que utiliza SVM como estimador base. El Bagging se configura para utilizar 500 estimadores y subconjuntos de datos de tamaño 100. Después de entrenar el clasificador Bagging, se imprime el puntaje out-of-bag, que es una estimación del rendimiento del modelo en datos no vistos durante el entrenamiento.

In [29]:
# Crear un clasificador Bagging con SVM como estimador base
bag_clf = BaggingClassifier(
    base_estimator=svm_clf,
    n_estimators=500,
    max_samples=100,
    max_features=1.0,
    bootstrap=True,
    oob_score=True,
    n_jobs=-1,
    random_state=0
)

# Entrenar el clasificador Bagging
bag_clf.fit(X_train, y_train)

# Imprimir el puntaje out-of-bag del clasificador Bagging
print("Out-of-Bag Score:", bag_clf.oob_score_)


c:\Users\enzor\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Out-of-Bag Score: 0.74876
